### Original

In [1]:
import altair as alt
from vega_datasets import data

source = data.barley()

alt.Chart(source).mark_bar().encode(
    x='year:O',
    y='sum(yield):Q',
    color='year:N',
    column='site:N'
)

alt.Chart(...)

In [2]:
import altair as alt
import pandas as pd
import json

# Datensatz laden
source = "Teildatensatz.json"
data_source = pd.DataFrame(pd.read_json(source))
# data_source

#Attribute aussortieren, die nicht mehr gebaucht werden bzw. die Attribute die gebraucht werden auswählen
data_attribute = data_source[[
    "timestamp", "location_id", "location_name",
    "ltr_label", "rtl_label",
    "weather_condition", "temperature",
    "pedestrians_count",
    "adult_ltr_pedestrians_count", "adult_rtl_pedestrians_count",
    "child_ltr_pedestrians_count", "child_rtl_pedestrians_count",
    "year", "month", "day"
]]
# data_attribute

# data_attribute["Datum"] = data_attribute["timestamp"].dt.date # ChatGPT: Extahiert das Datum vom Timestamp und speichert es in einem neuen Attribut
data_attribute["Stunde"] = data_attribute["timestamp"].dt.hour # ChatGPT: Extahiert die Stunde vom Timestamp und speichert es in einem neuen Attribut

data_attribute["Datum"] = (data_attribute["year"].astype(str).str.zfill(4) + 
                                  data_attribute["month"].astype(str).str.zfill(2) + 
                                  data_attribute["day"].astype(str).str.zfill(2)).astype(int)



loc_id = 329
data_filtered = data_attribute[data_attribute["location_id"] == loc_id] # Kann später in einem Dropdown ausgewählt werden

# data_filtered

#data_filtered_2 = data_filtered[data_filtered["Datum"] == pd.to_datetime("2021-10-15").date()] # Wird später mit einem Schiebegegler in der Grafik ausgewählt
#data_filtered_2


data_richtung = data_filtered.melt(
    id_vars=["Stunde", "Datum", "location_name", "location_id"],
    value_vars=["adult_ltr_pedestrians_count", "adult_rtl_pedestrians_count"], # Kann später zwischen Kinder und Erwachsene gewählt werden
    var_name="Richtung",
    value_name="Anzahl")
data_richtung

/var/folders/s1/90kv3cpx61v31kb62y63wm2c0000gn/T/ipykernel_21253/2299907240.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_attribute["Stunde"] = data_attribute["timestamp"].dt.hour # ChatGPT: Extahiert die Stunde vom Timestamp und speichert es in einem neuen Attribut
/var/folders/s1/90kv3cpx61v31kb62y63wm2c0000gn/T/ipykernel_21253/2299907240.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_attribute["Datum"] = (data_attribute["year"].astype(str).str.zfill(4) +


,Stunde,Datum,location_name,location_id,Richtung,Anzahl
0,0,20211015,Bahnhofstrasse (Mitte),329,adult_ltr_pedestrians_count,17
1,1,20211015,Bahnhofstrasse (Mitte),329,adult_ltr_pedestrians_count,12
2,2,20211015,Bahnhofstrasse (Mitte),329,adult_ltr_pedestrians_count,10
3,3,20211015,Bahnhofstrasse (Mitte),329,adult_ltr_pedestrians_count,52
4,4,20211015,Bahnhofstrasse (Mitte),329,adult_ltr_pedestrians_count,108
...,...,...,...,...,...,...
763,19,20250715,Bahnhofstrasse (Mitte),329,adult_rtl_pedestrians_count,332
764,20,20250715,Bahnhofstrasse (Mitte),329,adult_rtl_pedestrians_count,209
765,21,20250715,Bahnhofstrasse (Mitte),329,adult_rtl_pedestrians_count,95
766,22,20250715,Bahnhofstrasse (Mitte),329,adult_rtl_pedestrians_count,43


In [4]:
source = data_richtung

unique_dates = sorted(source["Datum"].unique().tolist())

date_intex_map = {d: i for i, d in enumerate(unique_dates)}
source["date_idx"] = source["Datum"].map(date_intex_map)

label_map = {"adult_ltr_pedestrians_count": "von Links nach Rechts", 
             "adult_rtl_pedestrians_count": "von Rechts nach Links"}
source["Richtung_Label"] = source["Richtung"].map(label_map)

slider = alt.binding_range(min=0, max=len(unique_dates) - 1, step=1, name="Datum: ")
idx_param= alt.param(name="date_idx", value=0, bind=slider)

chart = alt.Chart(source).mark_bar().encode(
    x=alt.X("Stunde:O", title="Stunden"),
    xOffset='Richtung:N',
    y=alt.Y('Anzahl:Q', title="Anzahl Personen", scale=alt.Scale(domain=[0, 7000])),
    color=alt.Color('Richtung_Label:N', 
                    title="Richtung", 
                    scale=alt.Scale(domain=["von Links nach Rechts", "von Rechts nach Links"], range=["#D3AF63", "#5AB4AC"])),
    tooltip=["Datum:O", 
             'Stunde:O', 
             'Richtung_Label:N', 
             'Anzahl:Q']
).transform_filter(alt.datum.date_idx == idx_param
).add_params(idx_param
).properties(
        width="container",
        height=600
)
chart


alt.Chart(...)

In [6]:
chart.to_json()
with open("aurelias-chart_neu.json", "w") as f:
    f.write(chart.to_json())